In [41]:
from io import BytesIO
from zipfile import ZipFile, is_zipfile
from urllib.request import urlopen
from pathlib import Path
import pandas as pd
import requests
from prefect import flow, task
from prefect_gcp.cloud_storage import GcsBucket
from google.cloud import storage
import gcsfs
import json
import gzip
import io

In [42]:
# # @task()
# def read_gcs_bucket(connection, bucket_name: str) -> list:
#     """Get list of zip folders in GCS bucket"""
#     lsblob = connection.ls(f'{bucket_name}/zip')
#     # lsblob = [l.rsplit('/', 1)[-1] for l in lsblob] #only return the filename from the blob
#     print(lsblob)
    
#     return lsblob

In [43]:
# # @task()
# def unzip_blob(connection, source_path: str
#                 # , destination_path: str
#                 # , bucket
#                 ) -> list:
#     """Unzip folder"""

#     with connection.open(source_path, 'rb') as f:
#         print(f.readline())
        
        
#     #     g = gzip.GzipFile(fileobj=f)  # Decompress data with gzip
#     #     df = pd.read_csv(g)           # Read CSV file with Pandas

#     # print(df.head())

#     # blob = bucket.blob(source_path)
#     # tmp_dir = '../tmp'
#     # Path(tmp_dir).mkdir(parents=True, exist_ok=True)

#     # with open('../tmp/archive.zip', 'wb') as file_obj:
#     #     blob.download_to_file(file_obj)

#     # zip_archive = ZipFile('../tmp/archive.zip', 'r')
    
#     # destination_blob = bucket.blob(destination_path)
#     # zip_archive.extractall(destination_path)
    
#     # return zip_archive


In [44]:
# @task()
def convert_to_parquet():
    """Upload local parquet file to GCS"""


In [45]:
# # @flow(log_prints=True)
# def process_gcs_blob() -> None:
#     """Process blob in gcs bucket"""
#     file =  open("../creds.json", 'r') #json file for key
#     token = json.load(file) #convert json file to python dictionary
#     bucket_name = 'de_project_bucket'
#     fs = gcsfs.GCSFileSystem(project=' DTC DE Course'
#                             , token=token
#                             )

#     lsblob = read_gcs_bucket(fs, bucket_name)
#     source = lsblob[0]
#     df = unzip_blob(fs, source)
#     # lsfiles = unzip_blob(source, destination, bucket)

#     # storage_client = storage.Client.from_service_account_json("../creds.json")
#     # bucket_name = 'de_project_bucket' #parameterize this
#     # bucket = storage_client.get_bucket(bucket_name)

#     # destination = 'unzip/'
#     # lsblob = read_gcs_bucket(bucket)

#     # source = lsblob[0]
#     # lsfiles = unzip_blob(source, destination, bucket)

#     # for blob in lsblob:
#     #     lsfiles = unzip_blob(blob, bucket)

In [46]:
# @task()
def read_gcs_bucket(bucket) -> list:
    """Get list of zip folders in GCS bucket"""
    lsblob = list(bucket.list_blobs(prefix="zip"))
    lsblob = [l.name for l in lsblob] #only return the filename from the blob
    print(lsblob)
    
    return lsblob

In [47]:
# @task()
def unzip_blob(source_path: str, destination_path: str, bucket) -> list:
    """Unzip folder"""
    blob = bucket.blob(source_path)

    zipbytes = io.BytesIO(blob.download_as_string())

    if is_zipfile(zipbytes):
        with ZipFile(zipbytes, 'r') as myzip:
            for contentfilename in myzip.namelist():
                contentfile = myzip.read(contentfilename)
                print(contentfile)


    # tmp_dir = '../tmp'
    # Path(tmp_dir).mkdir(parents=True, exist_ok=True)

    # with open('../tmp/archive.zip', 'wb') as file_obj:
    #     blob.download_to_file(file_obj)

    # zip_archive = ZipFile('../tmp/archive.zip', 'r')
    
    # destination_blob = bucket.blob(destination_path)
    # zip_archive.extractall(destination_path)
    
    # return zip_archive


In [48]:
# @flow(log_prints=True)
def process_gcs_blob() -> None:
    """Process blob in gcs bucket"""
    storage_client = storage.Client.from_service_account_json("../creds.json")
    bucket_name = 'de_project_bucket' #parameterize this
    bucket = storage_client.get_bucket(bucket_name)

    destination = 'unzip/'
    lsblob = read_gcs_bucket(bucket)

    source = lsblob[0]
    lsfiles = unzip_blob(source, destination, bucket)

    # for blob in lsblob:
    #     lsfiles = unzip_blob(blob, bucket)

In [49]:
# if __name__ == '__main__':
process_gcs_blob()

['zip/ps_1981_1989_csv.zip', 'zip/ps_1990_1994_csv.zip', 'zip/ps_1995_1999_csv.zip', 'zip/ps_2000_2004_csv.zip', 'zip/ps_2005_2009_csv.zip', 'zip/ps_2010_2014_csv.zip', 'zip/ps_2015_2016_csv.zip', 'zip/ps_2017_csv.zip', 'zip/ps_2018_csv.zip', 'zip/ps_2019_csv.zip', 'zip/ps_2020_csv.zip', 'zip/ps_2021_csv.zip']
